In [1]:
pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 167.4 kB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.4 MB/s eta 0:00:000m eta 0:00:010:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 3.5 MB/s eta 0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 5.4 MB/s eta 0:00:000m eta 0:00:0136m0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 2.9 MB/s eta 0:00:000m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 288.9 kB/s eta 0:00:00 eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 1.9 MB/s eta 0:00:000m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 1.3 MB/s eta 0:00:000m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 6.0 MB

In [ ]:
"""
This setup leverages the strengths of both CNNs (for capturing local patterns) and 
BiLSTM with Attention (for capturing temporal dependencies and important features) to 
provide a robust solution for stock prediction. Adjust the input shape, hyperparameters, 
and data preprocessing steps as necessary for your specific dataset and prediction task.
"""

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv1D, BatchNormalization,
    Activation, Add, MaxPooling1D, Flatten, Dense, LSTM,
    Bidirectional, Attention, Concatenate
)
from tensorflow.keras.optimizers import Adam

In [5]:
# Define the CNN with residula block
def residual_block(x,filters,kernel_size=3):
    shortcut = x
    x = Conv1D(filters,kernel_size, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv1D(filters,kernel_size,padding="same")(x)
    x = BatchNormalization()(x)
    x = Add()([shortcut,x])
    x = Activation("relu")(x)
    return x

def create_cnn_model(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv1D(64,3,padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = MaxPooling1D(2)(x)
    x = residual_block(x,64)
    x = MaxPooling1D(2)(x)
    x = residual_block(x,64)
#     x = Flatten()(x)
#     outputs = Dense(1)(x)
#     model = Model(inputs,outputs)
    return x

In [6]:
# Define BiLSTM with Attention mechanism
def attention_mech_layer(inputs):
    q = Dense(128)(inputs)
    k = Dense(128)(inputs)
    v = Dense(128)(inputs)
    attention_output = Attention()([q,k,v])
    return attention_output

def create_bilstm_model(input_shape):
    inputs = Input(shape=input_shape)
    x = Bidirectional(LSTM(64,return_sequences=True))(inputs)
    x = attention_mech_layer(x)
    x = Flatten()(x)
    outputs = Dense(1)(x)
    model = Model(inputs,outputs)
    return model

In [7]:
def create_ensemble_model(input_shape):
    inputs = Input(shape=input_shape)
    cnn_model = create_cnn_model(input_shape)
    bilstm_model = create_bilstm_model(cnn_model)
    # check this part of the code
    x = Dense(128,activation="relu")(bilstm_model.output)
    x = Dense(64,activation="relu")(x)
    outputs = Dense(1)(x)
    model = Model(inputs=[inputs],outputs=outputs)
    return model

In [ ]:
def train():
    # load the data
    input_shape = (100,1)
    esemble_model = create_ensemble_model(input_shape)
    esemble_model.compile(optimizer=Adam(learning_rate=0.001),loss='mse', metrics=['mae'])
    
    ensemble_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)
    
    # evaluate model
    ensemble_model.evaluate(X_test, y_test)
    print(f'Test Loss: {loss}, Test MAE: {mae}')
    
    predictions = ensemble_model.predict(X_test)
    

def evaluate_model(X_test,ytest):
    return ensemble_model.evaluate(X_test, y_test)

In [ ]:
def create_cnn_bilstm_model(input_shape, conv_filters, lstm_units, dense_units,learning_rate):
    inputs = Input(shape=input_shape)
    
    #CNN part
    x = Conv1D(filters=conv_filters,kernel_size=3,padding="same", activate="relu")(inputs)
    x=BatchNormalization()(x)
    x = MaxPooling1D(pool_size=2)(x)
    
    # BiLSTM part
    x= Bidirectional(LSTM(units=lstm_units,return_sequences=TRue))(x)
    x = Flatten()(x)
    
    # Dense part
    x = Dense(units=dense_units,activation="relu")(x)
    outputs = Dense(1)(x)
    model = Model(inputs,outputs)
    model.compile(optimizer=Adam(learning_rate=learning_rate),loss='mse',metrics=['mae'])
    return model
    

# Define fitness function
def fitness_function(hyperparams,input_shape,X_train,ytrain,x_val,y_val):
    conv_filters, lstm_units,dense_units,learning_rate = hyperparams
    model = create_cnn_bilstm_model(input_shape,conv_filters, lstm_units,dense_units, learning_rate)
    
    model.fit(X_train,ytrain,epoch=10,batch_size=32,validation_split=0.2,verbose=0)
    loss, mae = model.evaluate(x_val,y_val,verbose=0)
    
    return mae


# Red deer optimization algorithm
def red_deer_optimization(pop_size,num_generations,search_space,fitness_function):
    # Initialize the population
    population = np.random.uniform(low=search_space["low"],high=search_space["high"],size = (pop_size,len(search_space["low"])))
    
    best_solution = None
    best_fitness = float('inf')
    
    for generation in range(num_generations):
        # evaluate the fitness of each solution
        fitness_values = np.array([fitness_function(ind) for ind in population])
        min_fitness_idx = np.argmin(fitness_values)
        if fitness_values[min_fitness_idx] < best_fitness:
            best_fitness = fitness_values[min_fitness_idx]
            best_solution = population[min_fitness_idx]
        
        # apply RDO algorithm steps (simplified for illustration)
        new_population = []
        for i in range(pop_size//2):
            male = population[i]
            female = population[pop_size // 2 +i]
            
            # Roaring: small mutation
            male_offspring = male + np.random.normal(0,0.1,size=male.shape)
            female_offspring = female + np.random.normal(0,0.1,size=female.shape)
            
            # Mate crossover
            offspring = (male+female)/2
            
            # Add new individuals to population
            new_population.extend([male_offspring,female_offspring,offspring])
        
        #Select the new population for the next generation
        population = np.array(new_population[:pop_size])
    return best_solution, best_fitness


"""
search_space = {
    'low': [16, 16, 16, 1e-5],  # Min values for conv_filters, lstm_units, dense_units, learning_rate
    'high': [128, 128, 128, 1e-2]  # Max values for conv_filters, lstm_units, dense_units, learning_rate
}

best_solution, best_fitness = red_deer_optimization(
    pop_size=20,
    num_generations=50,
    search_space=search_space,
    fitness_function=fitness_function
)

print(f'Best hyperparameters: {best_solution}')
print(f'Best validation MAE: {best_fitness}')
"""

In [ ]:
class RDA:
    def __init__(self, num_agents, max_iter, train_data,label)